# Staging file for data cleaning & pre-processing

In [89]:
import json
import pandas as pd
import re 
import ast
import datetime
from datetime import date

In [126]:
data_json = json.load(open('data.json', 'r'))
data_json

[{'log': '--------- beginning of main\n'},
 {'log': '--------- beginning of a new ride\n'},
 {'log': '2022-11-03 18:42:00.422392 mendoza v9: Getting user data from server..\n'},
 {'log': '2022-11-03 18:42:00.922427 mendoza v9: [SYSTEM] data = {"user_id":100,"name":"Steven Pearce","gender":"male","address":"0 Simpson locks,Wilkinsonshire,E68 8SH","date_of_birth":273715200000,"email_address":"steven.p@gmail.com","height_cm":167,"weight_kg":83,"account_create_date":1647993600000,"bike_serial":"SN0000","original_source":"direct"}\n'},
 {'log': '2022-11-03 18:42:01.422451 mendoza v9: [INFO]: Ride - duration = 1.0; resistance = 0\n'},
 {'log': '2022-11-03 18:42:01.922469 mendoza v9: [INFO]: Telemetry - hrt = 0; rpm = 0; power = 0.0\n'},
 {'log': '2022-11-03 18:42:02.422486 mendoza v9: [INFO]: Ride - duration = 2.0; resistance = 0\n'},
 {'log': '2022-11-03 18:42:02.922501 mendoza v9: [INFO]: Telemetry - hrt = 0; rpm = 0; power = 0.0\n'},
 {'log': '2022-11-03 18:42:03.422516 mendoza v9: [INFO]

In [154]:
user_df_columns=['user_id', 'first_name', 'last_name', 'gender', 'street', 'area', 'postcode', 'd_o_b', 'email', 'height', 'weight', 'account_creation_date']
user_df = pd.DataFrame(columns=user_df_columns)
ride_df = pd.DataFrame(columns=['user_id', 'start_time', 'serial_no', 'source', 'duration', 'resistance', 'heart_rate', 'rpm', 'power'])
user = []
ride = []
data_regex = re.compile(r'{[\s\S]*}')
numbers_regex = re.compile(r'\d\.?\d*')
for log in data_json:
    if 'Getting user data from server' in log['log']:
        start_time = log['log'].split(' ')[1]
    if 'data = ' in log['log']:
        data = data_regex.findall(log['log'])
        user_dict = ast.literal_eval(data[0])
        if user_dict['user_id'] not in user_df['user_id'].unique():
            name = user_dict['name'].split(' ')
            address = user_dict['address'].split(',')
            user = [user_dict['user_id'], name[0], name[1], user_dict['gender'], address[0], address[1], address[2], user_dict['date_of_birth'], user_dict['email_address'], user_dict['height_cm'], user_dict['weight_kg'], user_dict['account_create_date']]
            user_df.loc[len(user_df)] = user
    elif 'Ride -' in log['log']:
        ride_data = numbers_regex.findall(log['log'])
    elif 'Telemetry -' in log['log']:
        telemetry_data = numbers_regex.findall(log['log'])
        ride = [user_dict['user_id'], start_time, user_dict['bike_serial'], user_dict['original_source'], ride_data[-2], ride_data[-1], telemetry_data[-3], telemetry_data[-2], telemetry_data[-1]]
        ride_df.loc[len(ride_df)] = ride

In [155]:
user_df

,user_id,first_name,last_name,gender,street,area,postcode,d_o_b,email,height,weight,account_creation_date
0,100,Steven,Pearce,male,0 Simpson locks,Wilkinsonshire,E68 8SH,273715200000,steven.p@gmail.com,167,83,1647993600000
1,101,Mr,Aaron,male,Studio 7,Francesca ports,West Robin,-214617600000,mr.aaron@gmail.com,155,43,1648425600000
2,102,Colin,Brown,male,Flat 87,Gavin parkway,Lake Jeffrey,567388800000,colin.b@gmail.com,163,63,1647475200000
3,103,Dr,Lynn,female,7 Tracey lane,Lake Peterside,WF3M 9HS,526521600000,dr_l@hotmail.com,177,75,1645747200000
4,104,Angela,Elliott,female,894 Kirsty lane,Junemouth,B7 8JD,719366400000,angela_elliott99@hotmail.com,155,48,1643241600000
5,105,Norman,Gough,male,2 Cole shore,South Tracyberg,L0J 7ED,-83376000000,n.g90@gmail.com,185,99,1647561600000
6,106,Allan,Chambers,male,Flat 4,Harris mill,Ryantown,229478400000,allan.c@yahoo.com,160,48,1640822400000
7,107,Dennis,Wilson,male,880 Frank inlet,West Katiestad,N69 8YD,650592000000,dennis_wilson@hotmail.com,174,63,1644883200000
8,108,Louis,Cook,male,02 Grace lane,East Sheilaberg,G5 6GS,-531705600000,louis.c@gmail.com,176,58,1640131200000
9,109,Leon,Wood,male,Studio 16m,Taylor mills,North Leonardfurt,751507200000,leon_w@hotmail.com,179,70,1648339200000


Improvements:
* Remove name titles (Mr, Miss, etc.)
* Check if is real postcode.
* Correct dt values.

In [161]:
ride_df

,user_id,start_time,serial_no,source,duration,resistance,heart_rate,rpm,power
0,100,18:42:00.422392,SN0000,direct,1.0,0,0,0,0.0
1,100,18:42:00.422392,SN0000,direct,2.0,0,0,0,0.0
2,100,18:42:00.422392,SN0000,direct,3.0,30,0,0,0.0
3,100,18:42:00.422392,SN0000,direct,4.0,30,0,0,0.0
4,100,18:42:00.422392,SN0000,direct,5.0,30,0,0,0.0
...,...,...,...,...,...,...,...,...,...
49790,115,08:28:18.944740,SN0000,google ads,0,30,41,10,13260673
49791,115,08:28:18.944740,SN0000,google ads,0,30,0,36,8.207264682
49792,115,08:28:18.944740,SN0000,google ads,0,30,43,10,93612828
49793,115,08:28:18.944740,SN0000,google ads,0,30,0,39,9.347636596000001
